In [1]:
import numpy as np                           #importing required libraries
import pandas as pd
import re
from nltk.corpus import stopwords
from random import randrange
from itertools import islice


In [2]:
#loading the data

data=pd.read_csv('data/amazon_cells_labelled.txt' , delimiter='\t', names=['review', 'sentiment'])

In [3]:
data.head()


,review,sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     1000 non-null   object
 1   sentiment  1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [5]:
data['sentiment'].value_counts()

0    500
1    500
Name: sentiment, dtype: int64

In [6]:
def text_filtering(text):
    #stop_words = list(stopwords.words('english'))
    stop_words=[]
    text = text.replace('\t', '')
    text = text.replace('\n', '')
    text = text.replace('\r', '')
    for word in text.split(" "):
        if word in stop_words:
            text=text.replace(word,'')
    text = text.rstrip() 
    regular_expression = '[^a-zA-z\s]'
    text = re.sub(regular_expression, '', text)
    return text
text_filtering("")

''

In [7]:
train_data = data[:int(0.9*len(data))]
test_and_dev_data = data[int(0.9*len(data)):]


dev_data=test_and_dev_data[:int(0.5 * len(test_and_dev_data))]
test_data=test_and_dev_data[int(0.5 * len(test_and_dev_data)):]

print(train_data.info())

print()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     900 non-null    object
 1   sentiment  900 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 14.2+ KB
None



In [8]:
print(test_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 950 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50 non-null     object
 1   sentiment  50 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 932.0+ bytes
None


In [9]:
print(dev_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 900 to 949
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50 non-null     object
 1   sentiment  50 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 932.0+ bytes
None


In [10]:
train_data['sentiment'].value_counts()

1    458
0    442
Name: sentiment, dtype: int64

In [11]:
test_data['sentiment'].value_counts()

0    33
1    17
Name: sentiment, dtype: int64

In [12]:
dev_data['sentiment'].value_counts()

0    25
1    25
Name: sentiment, dtype: int64

In [13]:
X_train = train_data['review']
y_train = train_data['sentiment']
# print(X_train)
# print(y_train)

X_dev = dev_data['review']
y_dev = dev_data['sentiment']
# print(X_dev)
# print(y_dev)

X_test = test_data['review']
y_test = test_data['sentiment']
# print(X_test)
# print(y_test)

In [14]:

def cal_prior_probs(y_train):
    prior_probs = []
    count_0 = sum(y_train == 0)
    prior_probs.append(count_0 / y_train.size)  
    count_1 = sum(y_train == 1)
    prior_probs.append(count_1 / y_train.size)    
    return prior_probs
prior_probs=cal_prior_probs(y_train)
print("Negative review class' prior probability--",prior_probs[0] )
print("Positive review class' prior probability--",prior_probs[1] )

Negative review class' prior probability-- 0.4911111111111111
Positive review class' prior probability-- 0.5088888888888888


In [15]:
def build_word_dict(data):
    word_freq = {}
    all_words=[]
    omit=[]
    for sentence in data:
            words=[]
            sentence = text_filtering(sentence)
            words=sentence.split(' ')
            for word in set(words) :
                word = word.lower()
                if word not in word_freq.keys():
                    word_freq[word] = 1
                    all_words.append(word)
                else:
                    word_freq[word] += 1
#     for word in word_freq:
#         if word_freq[word] <5:
#             omit.append(word)

#     for word in omit:
#         word_freq.pop(word,None)
        
            

    return word_freq, all_words

    
       
# allwords_freq,allwords_train=build_word_dict(X_train)
# print("Total words--",len(allwords_train))


# train_pos_reviews=train_data.loc[train_data.sentiment==1].review
# pos_words_freq,pos_words=build_word_dict(train_pos_reviews)
# print("Positive words--",len(pos_words))


# train_neg_reviews=train_data.loc[train_data.sentiment==0].review
# neg_words_freq,neg_words=build_word_dict(train_neg_reviews)
# print("Negative words--",len(neg_words))

In [16]:
allwords_freq,allwords_train=build_word_dict(X_train)
for word in allwords_freq:
        print(word,"--",allwords_freq[word],"     Probability of",word,"  is =",allwords_freq[word]/len(train_data))
       

in -- 80      Probability of in   is = 0.08888888888888889
i -- 245      Probability of i   is = 0.2722222222222222
so -- 31      Probability of so   is = 0.034444444444444444
us -- 2      Probability of us   is = 0.0022222222222222222
me -- 21      Probability of me   is = 0.023333333333333334
is -- 189      Probability of is   is = 0.21
for -- 106      Probability of for   is = 0.11777777777777777
a -- 169      Probability of a   is = 0.18777777777777777
by -- 7      Probability of by   is = 0.0077777777777777776
here -- 4      Probability of here   is = 0.0044444444444444444
it -- 213      Probability of it   is = 0.23666666666666666
to -- 146      Probability of to   is = 0.1622222222222222
go -- 6      Probability of go   is = 0.006666666666666667
way -- 7      Probability of way   is = 0.0077777777777777776
converter -- 1      Probability of converter   is = 0.0011111111111111111
unless -- 2      Probability of unless   is = 0.0022222222222222222
the -- 365      Probability of th

bmw -- 1      Probability of bmw   is = 0.0011111111111111111
trouble -- 3      Probability of trouble   is = 0.0033333333333333335
person -- 1      Probability of person   is = 0.0011111111111111111
quiet -- 2      Probability of quiet   is = 0.0022222222222222222
hearing -- 1      Probability of hearing   is = 0.0011111111111111111
fairly -- 1      Probability of fairly   is = 0.0011111111111111111
series -- 2      Probability of series   is = 0.0022222222222222222
choice -- 3      Probability of choice   is = 0.0033333333333333335
home -- 3      Probability of home   is = 0.0033333333333333335
station -- 2      Probability of station   is = 0.0022222222222222222
docking -- 2      Probability of docking   is = 0.0022222222222222222
beautiful -- 2      Probability of beautiful   is = 0.0022222222222222222
d -- 1      Probability of d   is = 0.0011111111111111111
advertised -- 2      Probability of advertised   is = 0.0022222222222222222
dwrongly -- 1      Probability of dwrongly   is 

stupid -- 1      Probability of stupid   is = 0.0011111111111111111
cradles -- 1      Probability of cradles   is = 0.0011111111111111111
kits -- 1      Probability of kits   is = 0.0011111111111111111
excelent -- 1      Probability of excelent   is = 0.0011111111111111111
nicer -- 1      Probability of nicer   is = 0.0011111111111111111
noticed -- 2      Probability of noticed   is = 0.0022222222222222222
cingulair -- 1      Probability of cingulair   is = 0.0011111111111111111
era -- 1      Probability of era   is = 0.0011111111111111111
goes -- 3      Probability of goes   is = 0.0033333333333333335
hoursthe -- 1      Probability of hoursthe   is = 0.0011111111111111111
thereplacement -- 1      Probability of thereplacement   is = 0.0011111111111111111
dead -- 3      Probability of dead   is = 0.0033333333333333335
colored -- 1      Probability of colored   is = 0.0011111111111111111
cheaply -- 1      Probability of cheaply   is = 0.0011111111111111111
att -- 1      Probability of a

warranty -- 2      Probability of warranty   is = 0.0022222222222222222
luck -- 1      Probability of luck   is = 0.0011111111111111111
exchange -- 1      Probability of exchange   is = 0.0011111111111111111
refurb -- 1      Probability of refurb   is = 0.0011111111111111111
linksys -- 1      Probability of linksys   is = 0.0011111111111111111
something -- 2      Probability of something   is = 0.0022222222222222222
bar -- 2      Probability of bar   is = 0.0022222222222222222
placed -- 1      Probability of placed   is = 0.0011111111111111111
snug -- 1      Probability of snug   is = 0.0011111111111111111
keeps -- 1      Probability of keeps   is = 0.0011111111111111111
heavyit -- 1      Probability of heavyit   is = 0.0011111111111111111
falling -- 1      Probability of falling   is = 0.0011111111111111111
utter -- 1      Probability of utter   is = 0.0011111111111111111
promised -- 1      Probability of promised   is = 0.0011111111111111111
tiny -- 1      Probability of tiny   is = 

In [17]:
def fit(train_data,X_train):

    train_pos_reviews=train_data.loc[train_data.sentiment==1].review
    pos_words_freq,pos_words=build_word_dict(train_pos_reviews)
    train_neg_reviews=train_data.loc[train_data.sentiment==0].review
    neg_words_freq,neg_words=build_word_dict(train_neg_reviews)

    neg_words_freq.pop('', None)
    neg_condi_prob={}
    for word in neg_words_freq:
        print(word,"--",neg_words_freq[word],"      Conditional probability of",word," given sentiment is neg=",neg_words_freq[word]/len(train_neg_reviews))
        neg_condi_prob[word]=neg_words_freq[word]/len(train_neg_reviews) 
        
    pos_words_freq.pop('', None)
    pos_condi_prob={}
    for word in pos_words_freq:
        print(word,"--",pos_words_freq[word],"      Conditional probability of",word," given sentiment is pos=",pos_words_freq[word]/len(train_pos_reviews))
        pos_condi_prob[word]=pos_words_freq[word]/len(train_pos_reviews)
    
    
    return pos_condi_prob,neg_condi_prob
    
pos_condi_prob_train,neg_condi_prob_train=fit(train_data,X_train)   


in -- 49       Conditional probability of in  given sentiment is neg= 0.11085972850678733
i -- 120       Conditional probability of i  given sentiment is neg= 0.27149321266968324
so -- 13       Conditional probability of so  given sentiment is neg= 0.029411764705882353
us -- 1       Conditional probability of us  given sentiment is neg= 0.0022624434389140274
me -- 14       Conditional probability of me  given sentiment is neg= 0.03167420814479638
is -- 79       Conditional probability of is  given sentiment is neg= 0.17873303167420815
for -- 52       Conditional probability of for  given sentiment is neg= 0.11764705882352941
a -- 83       Conditional probability of a  given sentiment is neg= 0.18778280542986425
by -- 7       Conditional probability of by  given sentiment is neg= 0.01583710407239819
here -- 2       Conditional probability of here  given sentiment is neg= 0.004524886877828055
it -- 111       Conditional probability of it  given sentiment is neg= 0.251131221719457
to -- 7

but -- 23       Conditional probability of but  given sentiment is neg= 0.05203619909502263
motorolas -- 1       Conditional probability of motorolas  given sentiment is neg= 0.0022624434389140274
website -- 2       Conditional probability of website  given sentiment is neg= 0.004524886877828055
could -- 12       Conditional probability of could  given sentiment is neg= 0.027149321266968326
directions -- 1       Conditional probability of directions  given sentiment is neg= 0.0022624434389140274
again -- 5       Conditional probability of again  given sentiment is neg= 0.011312217194570135
went -- 5       Conditional probability of went  given sentiment is neg= 0.011312217194570135
misleading -- 1       Conditional probability of misleading  given sentiment is neg= 0.0022624434389140274
most -- 3       Conditional probability of most  given sentiment is neg= 0.006787330316742082
commercials -- 1       Conditional probability of commercials  given sentiment is neg= 0.0022624434389140274

felt -- 2       Conditional probability of felt  given sentiment is neg= 0.004524886877828055
crack -- 1       Conditional probability of crack  given sentiment is neg= 0.0022624434389140274
ever -- 12       Conditional probability of ever  given sentiment is neg= 0.027149321266968326
worst -- 12       Conditional probability of worst  given sentiment is neg= 0.027149321266968326
freezes -- 1       Conditional probability of freezes  given sentiment is neg= 0.0022624434389140274
frequently -- 1       Conditional probability of frequently  given sentiment is neg= 0.0022624434389140274
mostly -- 1       Conditional probability of mostly  given sentiment is neg= 0.0022624434389140274
looks -- 6       Conditional probability of looks  given sentiment is neg= 0.013574660633484163
embarrassing -- 1       Conditional probability of embarrassing  given sentiment is neg= 0.0022624434389140274
kind -- 3       Conditional probability of kind  given sentiment is neg= 0.006787330316742082
childlike

specially -- 1       Conditional probability of specially  given sentiment is neg= 0.0022624434389140274
face -- 2       Conditional probability of face  given sentiment is neg= 0.004524886877828055
going -- 3       Conditional probability of going  given sentiment is neg= 0.006787330316742082
save -- 2       Conditional probability of save  given sentiment is neg= 0.004524886877828055
ill -- 2       Conditional probability of ill  given sentiment is neg= 0.004524886877828055
along -- 2       Conditional probability of along  given sentiment is neg= 0.004524886877828055
ringing -- 2       Conditional probability of ringing  given sentiment is neg= 0.004524886877828055
reason -- 2       Conditional probability of reason  given sentiment is neg= 0.004524886877828055
starts -- 2       Conditional probability of starts  given sentiment is neg= 0.004524886877828055
drivng -- 1       Conditional probability of drivng  given sentiment is neg= 0.0022624434389140274
having -- 4       Conditiona

utter -- 1       Conditional probability of utter  given sentiment is neg= 0.0022624434389140274
timeframe -- 1       Conditional probability of timeframe  given sentiment is neg= 0.0022624434389140274
promised -- 1       Conditional probability of promised  given sentiment is neg= 0.0022624434389140274
spring -- 1       Conditional probability of spring  given sentiment is neg= 0.0022624434389140274
three -- 3       Conditional probability of three  given sentiment is neg= 0.006787330316742082
finally -- 1       Conditional probability of finally  given sentiment is neg= 0.0022624434389140274
four -- 1       Conditional probability of four  given sentiment is neg= 0.0022624434389140274
visor -- 1       Conditional probability of visor  given sentiment is neg= 0.0022624434389140274
latch -- 1       Conditional probability of latch  given sentiment is neg= 0.0022624434389140274
light -- 2       Conditional probability of light  given sentiment is neg= 0.004524886877828055
tungsten -- 1 

puff -- 1       Conditional probability of puff  given sentiment is neg= 0.0022624434389140274
son -- 1       Conditional probability of son  given sentiment is neg= 0.0022624434389140274
itmy -- 1       Conditional probability of itmy  given sentiment is neg= 0.0022624434389140274
nano -- 1       Conditional probability of nano  given sentiment is neg= 0.0022624434389140274
dissapointed -- 1       Conditional probability of dissapointed  given sentiment is neg= 0.0022624434389140274
reccommend -- 1       Conditional probability of reccommend  given sentiment is neg= 0.0022624434389140274
q -- 2       Conditional probability of q  given sentiment is neg= 0.004524886877828055
smartphone -- 1       Conditional probability of smartphone  given sentiment is neg= 0.0022624434389140274
wont -- 1       Conditional probability of wont  given sentiment is neg= 0.0022624434389140274
atleast -- 1       Conditional probability of atleast  given sentiment is neg= 0.0022624434389140274
lightweight -

s -- 2       Conditional probability of s  given sentiment is pos= 0.004366812227074236
unlike -- 1       Conditional probability of unlike  given sentiment is pos= 0.002183406113537118
cheap -- 3       Conditional probability of cheap  given sentiment is pos= 0.006550218340611353
perfectly -- 5       Conditional probability of perfectly  given sentiment is pos= 0.010917030567685589
fantastic -- 3       Conditional probability of fantastic  given sentiment is pos= 0.006550218340611353
colors -- 1       Conditional probability of colors  given sentiment is pos= 0.002183406113537118
wi -- 1       Conditional probability of wi  given sentiment is pos= 0.002183406113537118
superb -- 2       Conditional probability of superb  given sentiment is pos= 0.004366812227074236
same -- 2       Conditional probability of same  given sentiment is pos= 0.004366812227074236
charm -- 5       Conditional probability of charm  given sentiment is pos= 0.010917030567685589
sharp -- 3       Conditional proba

commuter -- 1       Conditional probability of commuter  given sentiment is pos= 0.002183406113537118
bose -- 1       Conditional probability of bose  given sentiment is pos= 0.002183406113537118
amazing -- 1       Conditional probability of amazing  given sentiment is pos= 0.002183406113537118
cancelling -- 1       Conditional probability of cancelling  given sentiment is pos= 0.002183406113537118
greatno -- 1       Conditional probability of greatno  given sentiment is pos= 0.002183406113537118
at -- 6       Conditional probability of at  given sentiment is pos= 0.013100436681222707
catching -- 1       Conditional probability of catching  given sentiment is pos= 0.002183406113537118
frog -- 1       Conditional probability of frog  given sentiment is pos= 0.002183406113537118
eye -- 1       Conditional probability of eye  given sentiment is pos= 0.002183406113537118
amazed -- 2       Conditional probability of amazed  given sentiment is pos= 0.004366812227074236
talk -- 1       Condit

wornout -- 1       Conditional probability of wornout  given sentiment is pos= 0.002183406113537118
ringer -- 1       Conditional probability of ringer  given sentiment is pos= 0.002183406113537118
choices -- 1       Conditional probability of choices  given sentiment is pos= 0.002183406113537118
upbeat -- 1       Conditional probability of upbeat  given sentiment is pos= 0.002183406113537118
pair -- 1       Conditional probability of pair  given sentiment is pos= 0.002183406113537118
crisp -- 2       Conditional probability of crisp  given sentiment is pos= 0.004366812227074236
smallest -- 1       Conditional probability of smallest  given sentiment is pos= 0.002183406113537118
provided -- 1       Conditional probability of provided  given sentiment is pos= 0.002183406113537118
stays -- 1       Conditional probability of stays  given sentiment is pos= 0.002183406113537118
clips -- 1       Conditional probability of clips  given sentiment is pos= 0.002183406113537118
video -- 2       C

In [18]:
#top positive words


dict1 = pos_condi_prob_train
sorted_dict = {}
sorted_keys = sorted(dict1, key=dict1.get,reverse=True)  

for w in sorted_keys:
    sorted_dict[w] = dict1[w]
 
sorted_dict.pop('i',None)
def take(n, iterable):
    
    return list(islice(iterable, n))

n_items_pos = take(10, sorted_dict.items())
print(n_items_pos)

[('the', 0.37554585152838427), ('and', 0.3231441048034934), ('is', 0.24017467248908297), ('it', 0.22270742358078602), ('this', 0.21397379912663755), ('great', 0.18995633187772926), ('a', 0.18777292576419213), ('phone', 0.1703056768558952), ('to', 0.14847161572052403), ('very', 0.14192139737991266)]


In [19]:
#top negative words


dict1 = neg_condi_prob_train
sorted_dict = {}
sorted_keys = sorted(dict1, key=dict1.get,reverse=True)  

for w in sorted_keys:
    sorted_dict[w] = dict1[w]
    
n_items_neg = take(10, sorted_dict.items())
print(n_items_neg)


[('the', 0.43665158371040724), ('i', 0.27149321266968324), ('it', 0.251131221719457), ('and', 0.2149321266968326), ('this', 0.19909502262443438), ('not', 0.19457013574660634), ('a', 0.18778280542986425), ('is', 0.17873303167420815), ('to', 0.17647058823529413), ('phone', 0.14027149321266968)]


In [20]:
def score(real, prediction):
    count = 0
    for i in range(len(real)):
        if real[i] == prediction[i]:
            count+=1
    accuracy=count/len(real)*100
    return accuracy

In [21]:
def pred(data,ans,pos_condi_prob,neg_condi_prob):
    y_preds=[]
    for sentence in data:
        sentence=text_filtering(sentence)
        pos_lik=1
        neg_lik=1
        for word in sentence.split(' '):
            word=word.lower()
            if word not in pos_condi_prob.keys():
                pos_condi_prob[word]=0
            if word not in neg_condi_prob.keys():
                neg_condi_prob[word]=0
            pos_lik*=pos_condi_prob[word]
            neg_lik*=neg_condi_prob[word]
        pos_posterior=pos_lik*prior_probs[1]
        neg_posterior=neg_lik*prior_probs[0]
        if pos_posterior>neg_posterior:
            y_preds.append(1)
        else:
            y_preds.append(0)
    
    return score(ans.tolist(),y_preds)
print("Accuracy on training data--",pred(X_train,y_train,pos_condi_prob_train,neg_condi_prob_train) )   
print("Accuracy on development data--",pred(X_dev,y_dev,pos_condi_prob_train,neg_condi_prob_train))
print("Accuracy on test data--",pred(X_test,y_test,pos_condi_prob_train,neg_condi_prob_train))

Accuracy on training data-- 93.33333333333333
Accuracy on development data-- 54.0
Accuracy on test data-- 74.0


In [22]:
def k_fold_acc(dataset):         # here k is 5
    
    dat_split=np.array_split(dataset,5)
    review_split=np.array_split(dataset['review'],5)
    senti_split=np.array_split(dataset['sentiment'],5)
    fold_acc=[]
    
    
    arrd1= np.concatenate((dat_split[1],dat_split[2],dat_split[3],dat_split[4]))
    arrx1=np.concatenate((review_split[1],review_split[2],review_split[3],review_split[4]))
    dfd1 = pd.DataFrame(arrd1, columns = ['review','sentiment'])
    dfx1= df = pd.DataFrame(arrx1, columns = ['review'])
    pos_condi_prob_dev,neg_condi_prob_dev=fit(dfd1,dfx1)
    acc=pred(review_split[0],senti_split[0],pos_condi_prob_dev,neg_condi_prob_dev)    
    fold_acc.append(acc)

    
    arrd2= np.concatenate((dat_split[0],dat_split[2],dat_split[3],dat_split[4]))
    arrx2=np.concatenate((review_split[0],review_split[2],review_split[3],review_split[4]))
    dfd2 = pd.DataFrame(arrd2, columns = ['review','sentiment'])
    dfx2= df = pd.DataFrame(arrx2, columns = ['review'])
    pos_condi_prob_dev,neg_condi_prob_dev=fit(dfd2,dfx2)
    acc=pred(review_split[1],senti_split[1],pos_condi_prob_dev,neg_condi_prob_dev)    
    fold_acc.append(acc)
    
    arrd3= np.concatenate((dat_split[0],dat_split[1],dat_split[3],dat_split[4]))
    arrx3=np.concatenate((review_split[0],review_split[1],review_split[3],review_split[4]))
    dfd3 = pd.DataFrame(arrd3, columns = ['review','sentiment'])
    dfx3= df = pd.DataFrame(arrx3, columns = ['review'])
    pos_condi_prob_dev,neg_condi_prob_dev=fit(dfd3,dfx3)
    acc=pred(review_split[2],senti_split[2],pos_condi_prob_dev,neg_condi_prob_dev)    
    fold_acc.append(acc)
    
    arrd4= np.concatenate((dat_split[0],dat_split[1],dat_split[2],dat_split[4]))
    arrx4=np.concatenate((review_split[0],review_split[1],review_split[2],review_split[4]))
    dfd4 = pd.DataFrame(arrd4, columns = ['review','sentiment'])
    dfx4= df = pd.DataFrame(arrx4, columns = ['review'])
    pos_condi_prob_dev,neg_condi_prob_dev=fit(dfd4,dfx4)
    acc=pred(review_split[3],senti_split[3],pos_condi_prob_dev,neg_condi_prob_dev)    
    fold_acc.append(acc)
    

    arrd5= np.concatenate((dat_split[0],dat_split[1],dat_split[2],dat_split[3]))
    arrx5=np.concatenate((review_split[0],review_split[1],review_split[2],review_split[3]))
    dfd5 = pd.DataFrame(arrd5, columns = ['review','sentiment'])
    dfx5= df = pd.DataFrame(arrx5, columns = ['review'])
    pos_condi_prob_dev,neg_condi_prob_dev=fit(dfd5,dfx5)
    acc=pred(review_split[4],senti_split[4],pos_condi_prob_dev,neg_condi_prob_dev)    
    fold_acc.append(acc)

    return fold_acc
        
        
   
    
    
l=k_fold_acc(dev_data)   
print(l)  
print("Average--", sum(l)/len(l))


out -- 4       Conditional probability of out  given sentiment is neg= 0.2
a -- 9       Conditional probability of a  given sentiment is neg= 0.45
always -- 1       Conditional probability of always  given sentiment is neg= 0.05
failed -- 1       Conditional probability of failed  given sentiment is neg= 0.05
it -- 12       Conditional probability of it  given sentiment is neg= 0.6
beep -- 1       Conditional probability of beep  given sentiment is neg= 0.05
then -- 2       Conditional probability of then  given sentiment is neg= 0.1
says -- 1       Conditional probability of says  given sentiment is neg= 0.05
cuts -- 1       Conditional probability of cuts  given sentiment is neg= 0.05
makes -- 2       Conditional probability of makes  given sentiment is neg= 0.1
and -- 5       Conditional probability of and  given sentiment is neg= 0.25
sound -- 1       Conditional probability of sound  given sentiment is neg= 0.05
signal -- 1       Conditional probability of signal  given sentiment 

christmas -- 1       Conditional probability of christmas  given sentiment is pos= 0.047619047619047616
some -- 1       Conditional probability of some  given sentiment is pos= 0.047619047619047616
mine -- 1       Conditional probability of mine  given sentiment is pos= 0.047619047619047616
about -- 1       Conditional probability of about  given sentiment is pos= 0.047619047619047616
had -- 1       Conditional probability of had  given sentiment is pos= 0.047619047619047616
family -- 1       Conditional probability of family  given sentiment is pos= 0.047619047619047616
rest -- 1       Conditional probability of rest  given sentiment is pos= 0.047619047619047616
bought -- 1       Conditional probability of bought  given sentiment is pos= 0.047619047619047616
clear -- 1       Conditional probability of clear  given sentiment is pos= 0.047619047619047616
install -- 1       Conditional probability of install  given sentiment is pos= 0.047619047619047616
otherwise -- 1       Conditional p

abovepretty -- 1       Conditional probability of abovepretty  given sentiment is neg= 0.05263157894736842
useless -- 1       Conditional probability of useless  given sentiment is neg= 0.05263157894736842
there -- 1       Conditional probability of there  given sentiment is neg= 0.05263157894736842
volume -- 1       Conditional probability of volume  given sentiment is neg= 0.05263157894736842
low -- 1       Conditional probability of low  given sentiment is neg= 0.05263157894736842
quite -- 1       Conditional probability of quite  given sentiment is neg= 0.05263157894736842
mic -- 1       Conditional probability of mic  given sentiment is neg= 0.05263157894736842
joke -- 1       Conditional probability of joke  given sentiment is neg= 0.05263157894736842
them -- 1       Conditional probability of them  given sentiment is neg= 0.05263157894736842
returning -- 1       Conditional probability of returning  given sentiment is neg= 0.05263157894736842
maintain -- 1       Conditional prob

In [23]:
def fit_with_smoothing(train_data,X_train):
    allwords_freq,allwords_train=build_word_dict(X_train)
    train_pos_reviews=train_data.loc[train_data.sentiment==1].review
    pos_words_freq,pos_words=build_word_dict(train_pos_reviews)
    train_neg_reviews=train_data.loc[train_data.sentiment==0].review
    neg_words_freq,neg_words=build_word_dict(train_neg_reviews)
    alpha=1
    neg_words_freq.pop('', None)
    neg_condi_prob={}
    for word in neg_words_freq:
        #print(word,"--",neg_words_freq[word],"      Conditional probability of",word," given sentiment is neg=",neg_words_freq[word]/len(train_neg_reviews))
        neg_condi_prob[word]=(neg_words_freq[word]+1)/(len(train_neg_reviews) +alpha*2 )
        
    pos_words_freq.pop('', None)
    pos_condi_prob={}
    for word in pos_words_freq:
        #print(word,"--",pos_words_freq[word],"      Conditional probability of",word," given sentiment is neg=",pos_words_freq[word]/len(train_pos_reviews))
        pos_condi_prob[word]=(pos_words_freq[word]+1)/(len(train_pos_reviews) + alpha*2)
    
    
    return pos_condi_prob,neg_condi_prob
    
pos_condi_prob,neg_condi_prob=fit_with_smoothing(train_data,X_train)    

In [24]:
def pred(data,ans):
    y_preds=[]
    for sentence in data:
        sentence=text_filtering(sentence)
        pos_lik=1
        neg_lik=1
        alpha=1
        for word in sentence.split(' '):
            word=word.lower()
            if word not in pos_condi_prob.keys():
                pos_condi_prob[word]=0
            if word not in neg_condi_prob.keys():
                neg_condi_prob[word]=0
            pos_lik*=pos_condi_prob[word]
            neg_lik*=neg_condi_prob[word]
        pos_posterior=pos_lik*prior_probs[1]
        neg_posterior=neg_lik*prior_probs[0]
        if pos_posterior>neg_posterior:
            y_preds.append(1)
        else:
            y_preds.append(0)
    
    return score(ans.tolist(),y_preds)
print("Accuracy on training data--",pred(X_train,y_train) )   
print("Accuracy on development data--",pred(X_dev,y_dev))
print("Accuracy on test data--",pred(X_test,y_test))

Accuracy on training data-- 93.33333333333333
Accuracy on development data-- 54.0
Accuracy on test data-- 74.0
